In [61]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [62]:
# load data
apple_df = pd.read_csv("data/clean/AAPL_feature_engineered.csv")


In [63]:
# Define the features and target variables
target = ['Close']

# Define the features
features = apple_df.drop(['Close', 'Date', 'Quarter'], axis=1).columns.tolist()

# Create arrays for the features and the response variable
X = apple_df[features].values
y = apple_df[target].values

# Split the dataset into training and test sets: 80% training, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [64]:
# LSTM expects input in the form of [samples, time steps, features] - 3D tensor (3D array) 

# 1 time step (previous day's data) for each sample
time_steps = 1 

X_train_lstm = X_train.reshape(X_train.shape[0], time_steps, X_train.shape[1])
X_test_lstm = X_test.reshape(X_test.shape[0], time_steps, X_test.shape[1])

# avoid this error ==> ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).
X_train_lstm = X_train_lstm.astype('float32')
X_test_lstm = X_test_lstm.astype('float32')

In [65]:
from keras.models import Sequential
from keras.layers import LSTM, Dense



# Initialize the LSTM model
model = Sequential()

# Add the LSTM layer with 50 units and input shape (time_steps, features)
model.add(LSTM(50, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))

# Add a dense layer with 1 unit.
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

In [66]:
# Train the model on the training data
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, shuffle=False)


Epoch 1/50


ValueError: in user code:

    File "/Users/anujthakkar/Documents/Purdue/Projects/Stock Market/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Users/anujthakkar/Documents/Purdue/Projects/Stock Market/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/anujthakkar/Documents/Purdue/Projects/Stock Market/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Users/anujthakkar/Documents/Purdue/Projects/Stock Market/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/Users/anujthakkar/Documents/Purdue/Projects/Stock Market/.venv/lib/python3.10/site-packages/keras/src/utils/trac

In [ ]:
#LSTM Prediction
y_pred= lstm.predict(X_test)
print(y_pred.shape)
print(y_test.shape)

# rescacle the data back to original form
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)


In [ ]:
import matplotlib.pyplot as plt


